# Practica 4. Identificación de palabras, frases y documentos similares
**Tecnologías de Lenguaje Natural**

*Luis Fernando Rodríguez Domínguez*

5BV1

*Ingeniería en Inteligencia Artificial*

Fecha última de modificación: 14 de mayo del 2025

## Finalidad del Programa

Este programa en Python, presentado como un Jupyter Notebook, tiene como objetivo principal desarrollar un sistema capaz de identificar y cuantificar la similitud entre palabras, frases y documentos. Para ello, se aplicarán diversas técnicas de Procesamiento de Lenguaje Natural (NLP), abarcando tanto enfoques semánticos basados en recursos léxicos estructurados (WordNet) como enfoques sintácticos y semánticos basados en representaciones vectoriales densas (embeddings como GloVe y BERT).

El flujo de trabajo incluye:
1.  La creación de un corpus a partir de las introducciones de cinco libros con temáticas similares obtenidos del Proyecto Gutenberg.
2.  La normalización de estos documentos, incluyendo segmentación, tokenización, etiquetado gramatical y otras técnicas de limpieza.
3.  El cálculo de similitud entre palabras (verbos y sustantivos más frecuentes) utilizando métricas de WordNet y embeddings GloVe.
4.  La extracción de frases representativas de cada documento y el cálculo de similitud entre ellas utilizando WordNet y embeddings BERT.
5.  Un análisis comparativo de los resultados obtenidos mediante los diferentes enfoques.

## Datos de entrada

Los principales datos de entrada manejados en este programa son:

*   **Archivos de Texto (`.txt`):** Se requieren 5 archivos de texto, cada uno conteniendo la introducción de un libro. Estos archivos deben estar ubicados en el mismo directorio que el notebook y nombrados siguiendo un patrón (ej. `doc1.txt`, `doc2.txt`, ..., `doc5.txt`) para que `PlaintextCorpusReader` los pueda identificar.
*   **Corpus NLTK:** Se crea un objeto `PlaintextCorpusReader` para manejar los documentos de texto como un corpus.
*   **Listas y Strings:** Para almacenar textos, oraciones, tokens, y resultados.
*   **Diccionarios:** Para almacenar frecuencias de palabras, embeddings, y documentos etiquetados.
*   **Synsets de WordNet:** Objetos especiales de NLTK que representan conjuntos de sinónimos y conceptos léxicos.
*   **Vectores Numéricos (Embeddings):** Arrays de NumPy que representan palabras o frases en un espacio vectorial, generados por GloVe y BERT.
*   **Modelos Pre-entrenados:**
    *   **GloVe:** Se utiliza el archivo `glove.6B.50d.txt` (Wikipedia 2014 + Gigaword 5, vectores de 50 dimensiones). Este archivo debe descargarse de [Stanford NLP GloVe](https://nlp.stanford.edu/projects/glove/) y colocarse en el directorio de trabajo.
    *   **BERT:** Se utiliza el modelo `bert-base-uncased` de Hugging Face, que se descarga automáticamente a través de la biblioteca `transformers`.

## Listado y Descripción de Funciones

A continuación, se listan las funciones principales desarrolladas en este notebook, con una breve descripción de su propósito. Las descripciones detalladas se encuentran junto a la definición de cada función.

*   `get_wordnet_pos(treebank_tag)`: Convierte etiquetas gramaticales del formato Penn Treebank al formato compatible con WordNet.
*   `normalizar_documento_wordnet(texto_crudo, lematizador, stop_words_set)`: Procesa un texto crudo para obtener una lista de tuplas (lema, etiqueta_wordnet), adecuada para trabajar con WordNet.
*   `obtener_palabras_similares_wordnet(palabra_base, tipo_palabra_wn, top_n=5)`: Encuentra las `top_n` palabras más similares a `palabra_base` usando `path_similarity` y `wup_similarity` de WordNet.
*   `extraer_frase_representativa_textrank(texto_documento, num_oraciones=1)`: Extrae la frase (u oraciones) más representativa(s) de un texto usando el algoritmo TextRank.
*   `doc_to_synsets_wordnet(document_text, lematizador, stop_words_set)`: Convierte un texto en una lista de synsets de WordNet, filtrando y lematizando.
*   `path_similarity_entre_synsets(lista_synsets1, lista_synsets2)`: Calcula la similitud promedio basada en `path_similarity` entre dos listas de synsets.
*   `document_path_similarity_wordnet(doc_text1, doc_text2, lematizador, stop_words_set)`: Calcula la similitud entre dos documentos usando `path_similarity` sobre sus synsets.
*   `cargar_glove_model(glove_file_path)`: Carga los embeddings de GloVe desde un archivo.
*   `find_closest_glove_embeddings(palabra, embeddings_dict, top_n=5)`: Encuentra las `top_n` palabras más similares a `palabra` usando embeddings GloVe y similitud coseno.
*   `get_bert_sentence_embedding(sentence, bert_tokenizer, bert_model)`: Obtiene el embedding de una frase usando un modelo BERT.


### Dependencias

In [1]:
!pip install nltk
!pip install numpy scipy scikit-learn
!pip install sumy
!pip install transformers torch
#!pip install gensim # Opcional, si se quiere usar gensim para cargar GloVe

In [2]:
#!pip install scipy --upgrade --force-reinstall

### Imports

In [3]:
import nltk
import os
import re
from collections import Counter
import numpy as np
from scipy.spatial.distance import cosine # Para similitud coseno
import matplotlib.pyplot as plt # Para visualizaciones futuras, si se añaden

# NLTK (Natural Language Toolkit)
from nltk.corpus import PlaintextCorpusReader, stopwords, wordnet as wn
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Sumy (para resumen con TextRank)
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as SumyTokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Transformers (para BERT)
from transformers import BertTokenizer, BertModel
import torch

### Descarga de recursos adicionales necesarios

In [4]:
print("[INFO] Descargando recursos de NLTK necesarios...")

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4') # Open Multilingual Wordnet

print("[SUCCESS] Recursos de NLTK verificados/descargados.")

[INFO] Descargando recursos de NLTK necesarios...
[SUCCESS] Recursos de NLTK verificados/descargados.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## 1. Generación de cuerpo de documentos (0 pts)

El primer paso consiste en generar un corpus de documentos. Para esta práctica, se seleccionarán las **introducciones** de 5 libros del portal [Project Gutenberg](https://www.gutenberg.org/). Es importante que los libros pertenezcan a géneros o temas similares para que el análisis de similitud sea más significativo.

**Instrucciones para el alumno:**
1.  Visita [Project Gutenberg](https://www.gutenberg.org/).
2.  Identifica 5 libros de un género o tema de tu interés (ej., ciencia ficción, filosofía, historia de la música, etc.).
3.  Para cada libro, copia el texto correspondiente a su **introducción** (o prefacio, prólogo, o las primeras páginas si no hay una introducción formal).
4.  Guarda cada introducción en un archivo de texto plano (`.txt`) separado. Nombra los archivos como `doc1.txt`, `doc2.txt`, `doc3.txt`, `doc4.txt`, y `doc5.txt`.
5.  Coloca estos 5 archivos en el mismo directorio donde se encuentra este Jupyter Notebook.

Una vez que los archivos estén listos, el siguiente código los cargará utilizando `PlaintextCorpusReader` de NLTK.

In [5]:
corpus_directory = '/content/'
file_pattern = r'doc[1-5]\.txt' # Expresión regular para encontrar los archivos

# --- Inicialización del Tokenizer de Oraciones ---
# Se recomienda PunktSentenceTokenizer para una segmentación de oraciones robusta.
sentence_tokenizer = PunktSentenceTokenizer()

# --- Creación del Corpus ---
# PlaintextCorpusReader permite leer una colección de archivos de texto plano.
try:
    corpus = PlaintextCorpusReader(
        corpus_directory,
        file_pattern,
        sent_tokenizer=sentence_tokenizer,
        encoding='utf-8' # Especificar encoding por si acaso
    )
    print(f"[SUCCESS] Corpus cargado exitosamente desde '{corpus_directory}'.")
    print(f"Archivos identificados en el corpus: {corpus.fileids()}")

    # --- Verificación Inicial del Contenido ---
    if not corpus.fileids():
        print("[ERROR] No se encontraron archivos en el corpus. Verifica los nombres y la ubicación de tus archivos .txt.")
    else:
        print("\n--- Evidencia de Carga: Primera oración de cada documento ---")
        for file_id in corpus.fileids():
            try:
                first_sentence = corpus.sents(file_id)[0]
                print(f"Documento '{file_id}': {' '.join(first_sentence)[:100]}...") # Muestra los primeros 100 caracteres
            except IndexError:
                print(f"Documento '{file_id}': No se pudo extraer la primera oración (¿archivo vacío o sin segmentar?).")
            except Exception as e:
                print(f"Documento '{file_id}': Error al procesar - {e}")

except Exception as e:
    print(f"[ERROR] Ocurrió un error al intentar cargar el corpus: {e}")
    print("Por favor, asegúrate de que los archivos .txt (doc1.txt, ..., doc5.txt) existen en el directorio actual.")
    corpus = None # Asegurar que corpus es None si falla la carga


[SUCCESS] Corpus cargado exitosamente desde '/content/'.
Archivos identificados en el corpus: ['doc1.txt', 'doc2.txt', 'doc3.txt', 'doc4.txt', 'doc5.txt']

--- Evidencia de Carga: Primera oración de cada documento ---
Documento 'doc1.txt': It may be interesting to some persons to learn how it came about that Vatsyayana was first brought t...
Documento 'doc2.txt': Leopold von Sacher - Masoch was born in Lemberg , Austrian Galicia , on January 27 , 1836 ....
Documento 'doc3.txt': Son of a merchant , Boccaccio di Chellino di Buonaiuto , of Certaldo in Val d ' Elsa , a little town...
Documento 'doc4.txt': The amatory motif is pervasive , timeless , and universal ....
Documento 'doc5.txt': In reading several dozen books on sex matters for the young with a view to selecting the best for my...


## 2. Normalización de documentos (10 pts)

La normalización es un paso crucial en NLP para reducir el ruido y estandarizar el texto, lo que mejora la calidad de los análisis posteriores. En esta sección, cada documento se segmentará en oraciones y luego en tokens. A cada token se le asignará su categoría gramatical (Part-of-Speech, POS).

**Justificación de las Técnicas de Normalización:**

Las técnicas de normalización se aplicarán según las necesidades de cada tarea (puntos 3 a 6):

*   **Para tareas basadas en WordNet (Puntos 3 y 4):**
    *   **Tokenización:** Dividir el texto en palabras individuales.
    *   **Conversión a Minúsculas:** Estandariza las palabras (ej., "Verbo" y "verbo" se tratan igual).
    *   **Eliminación de Puntuación y No Alfabéticos:** Los signos de puntuación y caracteres no alfabéticos generalmente no aportan significado léxico para WordNet.
    *   **Eliminación de Stopwords:** Palabras comunes (ej., "el", "es", "un") que pueden no ser significativas para análisis semántico y pueden eliminarse para reducir el ruido.
    *   **POS Tagging:** Identificar la categoría gramatical de cada palabra (sustantivo, verbo, adjetivo, etc.). Esto es esencial para WordNet, ya que los synsets son específicos de una categoría gramatical.
    *   **Lematización:** Reducir las palabras a su forma base o lema (ej., "corriendo" -> "correr"). WordNet trabaja con lemas.

*   **Para tareas basadas en Embeddings (Puntos 5 y 6):**
    *   **GloVe (Punto 5):**
        *   **Tokenización y Minúsculas:** GloVe suele estar pre-entrenado con texto en minúsculas.
        *   La eliminación de stopwords y puntuación depende de cómo fue entrenado el modelo GloVe. Los modelos GloVe estándar suelen incluir stopwords y cierta puntuación, por lo que puede ser beneficioso mantenerlos si están en el vocabulario del modelo. Para este ejercicio, mantendremos un preprocesamiento similar al de WordNet (minúsculas, alfabéticos) para los términos objetivo (verbos más frecuentes) y luego buscaremos estos términos limpios en GloVe.
    *   **BERT (Punto 6):**
        *   BERT utiliza su propio **tokenizer especializado** (ej., WordPiece). Este tokenizer maneja la segmentación en sub-palabras, mayúsculas/minúsculas (si el modelo es "uncased"), y puntuación. Por lo tanto, a BERT se le deben pasar las frases con un preprocesamiento mínimo (idealmente, las oraciones originales).

**Implementación de la Normalización General:**

Primero, definiremos funciones auxiliares y realizaremos un pre-procesamiento que será la base para las tareas.


In [6]:
lemmatizer = WordNetLemmatizer()
stop_words_english = set(stopwords.words('english'))

def get_wordnet_pos(treebank_tag):
    """
    Convierte una etiqueta POS del formato Penn Treebank (usado por nltk.pos_tag)
    al formato compatible con WordNetLemmatizer y WordNet synsets.
    """
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None # Para otras etiquetas como determinantes, preposiciones, etc.

def normalizar_documento_wordnet(texto_crudo, lematizador_obj, stop_words_set):
    """
    Normaliza un texto para análisis con WordNet.
    Tokeniza, convierte a minúsculas, filtra no alfabéticos y stopwords,
    realiza POS tagging y lematiza.

    Args:
        texto_crudo (str): El texto original del documento.
        lematizador_obj (WordNetLemmatizer): Instancia del lematizador.
        stop_words_set (set): Conjunto de stopwords a eliminar.

    Returns:
        list: Una lista de tuplas (lema, etiqueta_wordnet) para cada palabra procesada.
    """
    tokens_procesados = []
    # Segmentar en oraciones y luego en palabras para un POS tagging más contextual
    for sent in nltk.sent_tokenize(texto_crudo):
        words = nltk.word_tokenize(sent)
        tagged_words = nltk.pos_tag(words)

        for word, tag in tagged_words:
            word_lower = word.lower()
            # Filtrar stopwords y no alfabéticos ANTES de lematizar
            if word_lower.isalpha() and word_lower not in stop_words_set:
                wn_tag = get_wordnet_pos(tag)
                if wn_tag: # Solo lematizar si es N, V, ADJ, ADV
                    lema = lematizador_obj.lemmatize(word_lower, pos=wn_tag)
                    tokens_procesados.append((lema, wn_tag))
    return tokens_procesados

# --- Procesamiento de cada documento del corpus ---
documentos_normalizados_wordnet = {} # Almacenará los tokens lematizados y etiquetados para WordNet

if corpus:
    print("\n--- Normalizando documentos para análisis con WordNet ---")
    for file_id in corpus.fileids():
        print(f"Procesando '{file_id}'...")
        texto_original = corpus.raw(file_id)
        documentos_normalizados_wordnet[file_id] = normalizar_documento_wordnet(texto_original, lemmatizer, stop_words_english)

        # Evidencia de normalización (primeros 10 tokens)
        if documentos_normalizados_wordnet[file_id]:
             print(f"Primeros 10 tokens normalizados para '{file_id}': {documentos_normalizados_wordnet[file_id][:10]}")
        else:
            print(f"'{file_id}' no produjo tokens normalizados (¿contenido filtrado completamente?).")
    print("[SUCCESS] Normalización para WordNet completada.")
else:
    print("[ERROR] El corpus no está cargado. No se puede proceder con la normalización.")


--- Normalizando documentos para análisis con WordNet ---
Procesando 'doc1.txt'...
Primeros 10 tokens normalizados para 'doc1.txt': [('interest', 'v'), ('person', 'n'), ('learn', 'v'), ('come', 'v'), ('vatsyayana', 'n'), ('first', 'r'), ('bring', 'v'), ('light', 'n'), ('translate', 'v'), ('english', 'a')]
Procesando 'doc2.txt'...
Primeros 10 tokens normalizados para 'doc2.txt': [('leopold', 'a'), ('von', 'n'), ('bear', 'v'), ('lemberg', 'n'), ('austrian', 'n'), ('galicia', 'n'), ('january', 'n'), ('study', 'v'), ('jurisprudence', 'n'), ('prague', 'n')]
Procesando 'doc3.txt'...
Primeros 10 tokens normalizados para 'doc3.txt': [('son', 'n'), ('merchant', 'n'), ('boccaccio', 'n'), ('chellino', 'n'), ('buonaiuto', 'n'), ('certaldo', 'n'), ('val', 'n'), ('little', 'a'), ('town', 'n'), ('midway', 'n')]
Procesando 'doc4.txt'...
Primeros 10 tokens normalizados para 'doc4.txt': [('amatory', 'n'), ('motif', 'n'), ('pervasive', 'a'), ('timeless', 'n'), ('universal', 'n'), ('phase', 'n'), ('manif

## 3. Similitud de palabras con synsets (10 pts)

En esta sección, para cada documento:
1.  Se identificará el **verbo más común** y el **sustantivo más frecuente**.
2.  Se hallarán los 5 términos más parecidos a ese verbo (y luego al sustantivo) usando dos métricas de similitud de WordNet:
    *   `wup_similarity` (Wu-Palmer similarity)
    *   `path_similarity`
3.  En total, se obtendrán 10 verbos/sustantivos similares por cada verbo/sustantivo frecuente (5 con cada métrica).

La lematización realizada en el paso anterior es crucial aquí, ya que WordNet opera sobre lemas.

In [7]:
def obtener_palabras_similares_wordnet(palabra_base, tipo_palabra_wn, top_n=5):
    """
    Encuentra las 'top_n' palabras más similares a 'palabra_base' de un 'tipo_palabra_wn'
    específico (ej. wn.VERB, wn.NOUN) usando path_similarity y wup_similarity de WordNet.

    Args:
        palabra_base (str): El lema de la palabra para la cual buscar similares.
        tipo_palabra_wn (str): El tipo de palabra según WordNet (wn.VERB, wn.NOUN, etc.).
        top_n (int): El número de palabras similares a retornar por cada métrica.

    Returns:
        tuple: Dos listas de tuplas (palabra_similar, similitud).
               La primera lista es para path_similarity, la segunda para wup_similarity.
               Cada tupla contiene el lema de la palabra similar y su puntuación de similitud.
    """
    similares_path = []
    similares_wup = []

    # Obtener el primer synset de la palabra base (el más común)
    synsets_base = wn.synsets(palabra_base, pos=tipo_palabra_wn)
    if not synsets_base:
        # print(f"Advertencia: No se encontraron synsets para '{palabra_base}' como {tipo_palabra_wn}.")
        return [], []

    synset_base_principal = synsets_base[0]

    # Comparar con todos los otros synsets del mismo tipo de palabra
    for synset_candidato in wn.all_synsets(pos=tipo_palabra_wn):
        if synset_candidato == synset_base_principal:
            continue # No comparar consigo mismo

        # Calcular similitudes
        path_sim = synset_base_principal.path_similarity(synset_candidato)
        wup_sim = synset_base_principal.wup_similarity(synset_candidato)

        # Almacenar si la similitud es válida
        if path_sim is not None:
            # Un synset puede tener múltiples lemas, tomamos el primero como representativo
            for lema in synset_candidato.lemmas():
                similares_path.append(((lema.name(), synset_candidato.name()), path_sim))
                break # Solo el primer lema del synset candidato

        if wup_sim is not None:
            for lema in synset_candidato.lemmas():
                similares_wup.append(((lema.name(), synset_candidato.name()), wup_sim))
                break # Solo el primer lema del synset candidato

    # Eliminar duplicados por nombre de lema, conservando la mayor similitud
    # (puede haber múltiples synsets para un mismo lema, o lemas que dan el mismo nombre)
    def unique_sorted_lemmas(sim_list):
        lemma_sim_dict = {}
        for (lemma_name, _), sim_score in sim_list:
            if lemma_name not in lemma_sim_dict or sim_score > lemma_sim_dict[lemma_name]:
                if lemma_name != palabra_base: # Excluir la palabra base de sus propios similares
                     lemma_sim_dict[lemma_name] = sim_score

        sorted_lemmas = sorted(lemma_sim_dict.items(), key=lambda item: item[1], reverse=True)
        return sorted_lemmas

    similares_path_unicos = unique_sorted_lemmas(similares_path)
    similares_wup_unicos = unique_sorted_lemmas(similares_wup)

    return similares_path_unicos[:top_n], similares_wup_unicos[:top_n]




### Procesamiento para cada documento

In [8]:
if corpus and documentos_normalizados_wordnet:
    print("\n--- Calculando Similitud de Palabras con Synsets ---")
    for file_id in corpus.fileids():
        print(f"\n--- Documento: {file_id} ---")
        tokens_doc = documentos_normalizados_wordnet.get(file_id, [])
        if not tokens_doc:
            print("No hay tokens normalizados para este documento.")
            continue

        # --- Identificar Verbo Más Común ---
        verbos_doc = [lema for lema, tag in tokens_doc if tag == wn.VERB]
        if verbos_doc:
            contador_verbos = Counter(verbos_doc)
            verbo_mas_comun, freq_verbo = contador_verbos.most_common(1)[0]
            print(f"Verbo más común: '{verbo_mas_comun}' (Frecuencia: {freq_verbo})")

            path_sim_verbos, wup_sim_verbos = obtener_palabras_similares_wordnet(verbo_mas_comun, wn.VERB)
            print("  Verbos similares (path_similarity):")
            for lema, sim in path_sim_verbos: # Ignoramos el synset name en la impresión
              print(f"    - {lema}: {sim:.4f}")
            if not path_sim_verbos: print("    No se encontraron verbos similares con path_similarity.")

            print("  Verbos similares (wup_similarity):")
            for lema, sim in wup_sim_verbos:
              print(f"    - {lema}: {sim:.4f}")
            if not wup_sim_verbos: print("    No se encontraron verbos similares con wup_similarity.")
        else:
            print("No se encontraron verbos en este documento.")

        # --- Identificar Sustantivo Más Común ---
        sustantivos_doc = [lema for lema, tag in tokens_doc if tag == wn.NOUN]
        if sustantivos_doc:
            contador_sustantivos = Counter(sustantivos_doc)
            sustantivo_mas_comun, freq_sust = contador_sustantivos.most_common(1)[0]
            print(f"\nSustantivo más común: '{sustantivo_mas_comun}' (Frecuencia: {freq_sust})")

            path_sim_sust, wup_sim_sust = obtener_palabras_similares_wordnet(sustantivo_mas_comun, wn.NOUN)
            print("  Sustantivos similares (path_similarity):")
            for lema, sim in path_sim_sust:
              print(f"    - {lema}: {sim:.4f}")
            if not path_sim_sust: print("    No se encontraron sustantivos similares con path_similarity.")

            print("  Sustantivos similares (wup_similarity):")
            for lema, sim in wup_sim_sust:
              print(f"    - {lema}: {sim:.4f}")
            if not wup_sim_sust: print("    No se encontraron sustantivos similares con wup_similarity.")
        else:
            print("No se encontraron sustantivos en este documento.")
else:
    print("[INFO] No se puede proceder con la similitud de palabras con synsets. Corpus o documentos normalizados no disponibles.")


--- Calculando Similitud de Palabras con Synsets ---

--- Documento: doc1.txt ---
Verbo más común: 'call' (Frecuencia: 8)
  Verbos similares (path_similarity):
    - label: 0.5000
    - refer: 0.5000
    - dub: 0.5000
    - style: 0.5000
    - baptize: 0.5000
  Verbos similares (wup_similarity):
    - refer: 0.8571
    - dub: 0.8571
    - style: 0.8571
    - baptize: 0.8571
    - rename: 0.8571

Sustantivo más común: 'work' (Frecuencia: 11)
  Sustantivos similares (path_similarity):
    - action: 0.5000
    - service: 0.5000
    - wash: 0.5000
    - care: 0.5000
    - activity: 0.5000
  Sustantivos similares (wup_similarity):
    - action: 0.9333
    - service: 0.9333
    - wash: 0.9333
    - care: 0.9333
    - operation: 0.9333

--- Documento: doc2.txt ---
Verbo más común: 'make' (Frecuencia: 5)
  Verbos similares (path_similarity):
    - overdo: 0.5000
    - breathe: 0.3333
    - hold: 0.3333
    - blow: 0.3333
    - act: 0.3333
  Verbos similares (wup_similarity):
    - overdo: 0.6

## 4. Similitud de documentos con synsets (10 pts)

Para esta sección:
1.  Para cada libro, se extraerá la **frase más representativa** de su introducción. Se utilizará el algoritmo TextRank (a través de la biblioteca `sumy`), que fue una opción robusta en la Práctica 3.
2.  De los cinco libros, se elegirá uno (por ejemplo, el primero, `doc1.txt`) y se comparará su frase representativa con las de los otros cuatro libros.
3.  La similitud entre estas frases representativas se calculará utilizando `path_similarity` a nivel de documento (considerando las frases como mini-documentos).

In [9]:
def extraer_frase_representativa_textrank(texto_documento, num_oraciones=1):
    """
    Extrae la(s) frase(s) más representativa(s) de un texto usando TextRank.

    Args:
        texto_documento (str): El texto original del documento.
        num_oraciones (int): Número de oraciones a extraer como resumen.

    Returns:
        str: La frase o frases más representativas concatenadas.
             Retorna string vacío si no se puede generar resumen.
    """
    if not texto_documento.strip():
        return ""
    try:
        parser = PlaintextParser.from_string(texto_documento, SumyTokenizer("english"))
        summarizer = TextRankSummarizer()
        resumen_oraciones = summarizer(parser.document, num_oraciones)
        return " ".join([str(oracion) for oracion in resumen_oraciones])
    except Exception as e:
        print(f"Error al extraer frase con TextRank: {e}")
        return ""

# --- Funciones para similitud de documentos con Path Similarity (adaptadas de los ejemplos) ---

def doc_to_synsets_wordnet(document_text, lematizador_obj, stop_words_set):
    """
    Convierte un texto (documento/frase) en una lista de sus synsets de WordNet.
    Utiliza la normalización definida previamente (tokenización, POS, lematización).
    """
    tokens_normalizados = normalizar_documento_wordnet(document_text, lematizador_obj, stop_words_set)
    synsets_lista = []
    for lema, wn_tag in tokens_normalizados:
        # Tomamos el primer synset (más común) para cada lema
        ss = wn.synsets(lema, pos=wn_tag)
        if ss:
            synsets_lista.append(ss[0])
    return synsets_lista

def path_similarity_entre_synsets(lista_synsets1, lista_synsets2):
    """
    Calcula una puntuación de similitud entre dos listas de synsets.
    Para cada synset en lista1, encuentra el synset más similar en lista2
    (usando path_similarity) y promedia estas puntuaciones máximas.
    """
    puntuaciones_maximas = []
    for s1 in lista_synsets1:
        mejor_sim_para_s1 = 0.0
        for s2 in lista_synsets2:
            sim = s1.path_similarity(s2)
            if sim is not None and sim > mejor_sim_para_s1:
                mejor_sim_para_s1 = sim
        if mejor_sim_para_s1 > 0: # Solo considerar si hay alguna similitud
            puntuaciones_maximas.append(mejor_sim_para_s1)

    if not puntuaciones_maximas:
        return 0.0
    return sum(puntuaciones_maximas) / len(puntuaciones_maximas)

def document_path_similarity_wordnet(doc_text1, doc_text2, lematizador_obj, stop_words_set):
    """
    Calcula la similitud entre dos textos usando path_similarity en sus synsets.
    Es una medida simétrica: (sim(doc1, doc2) + sim(doc2, doc1)) / 2.
    """
    synsets1 = doc_to_synsets_wordnet(doc_text1, lematizador_obj, stop_words_set)
    synsets2 = doc_to_synsets_wordnet(doc_text2, lematizador_obj, stop_words_set)

    if not synsets1 or not synsets2:
        return 0.0 # Si uno de los documentos no tiene synsets, la similitud es 0.

    sim1_a_2 = path_similarity_entre_synsets(synsets1, synsets2)
    sim2_a_1 = path_similarity_entre_synsets(synsets2, synsets1)

    return (sim1_a_2 + sim2_a_1) / 2.0

# --- Extracción de frases representativas ---
frases_representativas = {}
if corpus:
    print("\n--- Extrayendo Frases Representativas con TextRank ---")
    for file_id in corpus.fileids():
        texto_original_doc = corpus.raw(file_id)
        frase_repr = extraer_frase_representativa_textrank(texto_original_doc, num_oraciones=1)
        frases_representativas[file_id] = frase_repr
        print(f"Documento '{file_id}': \"{frase_repr}\"")
else:
    print("[INFO] No se puede extraer frases representativas. Corpus no disponible.")


# --- Comparación de similitud de frases (doc1 vs otros) ---
if corpus and frases_representativas and len(corpus.fileids()) > 1:
    print("\n--- Calculando Similitud de Frases Representativas con Path Similarity (WordNet) ---")

    ids_archivos = corpus.fileids()
    id_base = ids_archivos[4] # Tomamos el primer documento como base
    frase_base = frases_representativas.get(id_base)

    if frase_base:
        print(f"Frase base (de '{id_base}'): \"{frase_base}\"")
        for i in range(1, len(ids_archivos)):
            id_comparar = ids_archivos[i]
            frase_comparar = frases_representativas.get(id_comparar)
            if frase_comparar:
                similitud = document_path_similarity_wordnet(frase_base, frase_comparar, lemmatizer, stop_words_english)
                print(f"  Similitud con frase de '{id_comparar}': {similitud:.4f}")
            else:
                print(f"  No hay frase representativa para '{id_comparar}'.")
    else:
        print(f"No se pudo obtener la frase base para '{id_base}'.")
else:
    print("[INFO] No se puede calcular similitud de documentos con synsets. Corpus o frases no disponibles.")


--- Extrayendo Frases Representativas con TextRank ---
Documento 'doc1.txt': "The date of the 'Jayamangla' is fixed between the tenth and thirteenth centuries A.D., because while treating of the sixty-four arts an example is taken from the 'Kávyaprakásha,' which was written about the tenth century A.D. Again, the copy of the commentary procured was evidently a transcript of a manuscript which once had a place in the library of a Chaulukyan king named Vishaladeva, a fact elicited from the following sentence at the end of it:—"
Documento 'doc2.txt': "By this is meant the desire on the part of the individual affected of desiring himself completely and unconditionally subject to the will of a person of the opposite sex, and being treated by this person as by a master, to be humiliated, abused, and tormented, even to the verge of death."
Documento 'doc3.txt': "Despite his complaints of the malevolence of his critics in the Proem to the Fourth Day of the Decameron, he had no lack of appreci

## 5. Similitud de palabras con “embedding” (10 pts)

En esta sección, se utilizará el modelo pre-entrenado de **GloVe “Wikipedia 2014 + Gigaword 5”** (específicamente, la versión con vectores de 50 dimensiones, `glove.6B.50d.txt`) para identificar, en cada documento, los 5 términos más similares al **verbo más frecuente** de ese mismo documento (identificado en el punto 3). La relación entre términos se calculará usando **similitud de coseno**.

**Instrucciones para el alumno:**
1.  Descarga los embeddings de GloVe "Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d vectors)". El archivo que necesitas es `glove.6B.50d.txt`.
    *   Link de descarga: [GloVe Project Page](https://nlp.stanford.edu/projects/glove/) (busca `glove.6B.zip`).
2.  Descomprime el archivo `glove.6B.zip`.
3.  Coloca el archivo `glove.6B.50d.txt` en el mismo directorio que este Jupyter Notebook.

In [10]:
def cargar_glove_model(glove_file_path, expected_dim=50):
    """
    Carga los embeddings de GloVe desde un archivo a un diccionario.

    Args:
        glove_file_path (str): Ruta al archivo de GloVe (ej. 'glove.6B.50d.txt').
        expected_dim (int): La dimensionalidad esperada de los vectores GloVe.

    Returns:
        dict: Un diccionario donde las claves son palabras y los valores son sus vectores embedding (NumPy array),
              o None si ocurre un error crítico como FileNotFoundError.
    """
    print(f"[INFO] Cargando modelo GloVe desde: {glove_file_path}...")
    embeddings_dict = {}
    lines_skipped = 0
    try:
        with open(glove_file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f):
                values = line.split()

                if not values: # Manejar líneas vacías
                    # print(f"[DEBUG] Line {line_num+1}: Empty line. Skipping.")
                    lines_skipped +=1
                    continue

                word = values[0]

                # Verificar si el número de valores es consistente con la dimensión esperada
                if len(values) != expected_dim + 1:
                    print(f"[WARNING] Line {line_num+1}: Unexpected number of values for word '{word}'. Expected {expected_dim+1}, got {len(values)}. Skipping line: '{line.strip()[:100]}...'")
                    lines_skipped +=1
                    continue
                try:
                    vector = np.asarray(values[1:], "float32")
                    embeddings_dict[word] = vector
                except ValueError as e:
                    # Esto atrapará el error "could not convert string to float"
                    print(f"[WARNING] Line {line_num+1}: Could not convert vector values to float for word '{word}'. Error: {e}. Skipping line: '{line.strip()[:100]}...'")
                    lines_skipped +=1
                    continue

        if lines_skipped > 0:
            print(f"[INFO] Se omitieron {lines_skipped} líneas durante la carga del modelo GloVe debido a problemas de formato.")

        if not embeddings_dict:
            print("[WARNING] No se cargaron embeddings. El diccionario está vacío. Verifica el archivo GloVe, la ruta y su contenido.")
        else:
            print(f"[SUCCESS] Modelo GloVe cargado. {len(embeddings_dict)} vectores de palabras.")

    except FileNotFoundError:
        print(f"[ERROR] Archivo GloVe no encontrado en '{glove_file_path}'.")
        print("Por favor, descarga 'glove.6B.50d.txt' y colócalo en el directorio correcto.")
        return None
    except Exception as e:
        print(f"[ERROR] Ocurrió un error general al cargar GloVe: {e}")
        return None
    return embeddings_dict


# Asegúrate de que 'glove.6B.50d.txt' está en el mismo directorio que este notebook, o proporciona la ruta correcta.
# Para Google Colab, primero debes subir el archivo.
glove_file = '/content/glove.6B.50d.txt' # Ajusta esta ruta si es necesario
glove_embeddings = cargar_glove_model(glove_file, expected_dim=50) # Se especifica la dimensión esperada

def find_closest_glove_embeddings(palabra_objetivo, embeddings_dictionary, top_n=5):
    """
    Encuentra las 'top_n' palabras más similares a 'palabra_objetivo' usando
    embeddings GloVe y similitud coseno.

    Args:
        palabra_objetivo (str): La palabra para la cual buscar similares.
        embeddings_dictionary (dict): Diccionario de palabras y sus embeddings GloVe.
        top_n (int): Número de palabras similares a retornar.

    Returns:
        list: Lista de tuplas (palabra_similar, similitud_coseno), ordenada por similitud.
    """
    if embeddings_dictionary is None:
        print("[ERROR] El diccionario de embeddings GloVe no está cargado.")
        return []

    palabra_objetivo_lower = palabra_objetivo.lower() # GloVe suele estar en minúsculas
    if palabra_objetivo_lower not in embeddings_dictionary:
        # print(f"Advertencia: La palabra '{palabra_objetivo_lower}' no se encuentra en el vocabulario de GloVe.")
        return []

    embedding_objetivo = embeddings_dictionary[palabra_objetivo_lower]
    similitudes = {}

    for word, embedding_candidato in embeddings_dictionary.items():
        if word == palabra_objetivo_lower:
            continue
        # Similitud coseno = 1 - distancia coseno
        sim_score = 1 - cosine(embedding_objetivo, embedding_candidato)
        similitudes[word] = sim_score

    palabras_mas_similares = sorted(similitudes.items(), key=lambda item: item[1], reverse=True)
    return palabras_mas_similares[:top_n]

# --- Procesamiento para cada documento (usando verbos más frecuentes del punto 3) ---
if corpus and documentos_normalizados_wordnet and glove_embeddings:
    print("\n--- Calculando Similitud de Palabras con Embeddings GloVe (Verbos más frecuentes) ---")
    for file_id in corpus.fileids():
        print(f"\n--- Documento: {file_id} ---")
        tokens_doc = documentos_normalizados_wordnet.get(file_id, [])
        if not tokens_doc:
            print("No hay tokens normalizados para este documento.")
            continue

        verbos_doc = [lema for lema, tag in tokens_doc if tag == wn.VERB]
        if verbos_doc:
            contador_verbos = Counter(verbos_doc)
            verbo_mas_comun, _ = contador_verbos.most_common(1)[0]
            print(f"Verbo más común (lema): '{verbo_mas_comun}'")

            similares_glove = find_closest_glove_embeddings(verbo_mas_comun, glove_embeddings, top_n=5)
            if similares_glove:
                print(f"  Los 5 términos más similares a '{verbo_mas_comun}' según GloVe (coseno):")
                for palabra, sim in similares_glove:
                    print(f"    - {palabra}: {sim:.4f}")
            else:
                print(f"  No se encontraron palabras similares en GloVe para '{verbo_mas_comun}' o no está en el vocabulario.")
        else:
            print("No se encontraron verbos en este documento para analizar con GloVe.")
else:
    print("\n[INFO] No se puede proceder con similitud de palabras GloVe. Corpus, documentos normalizados o GloVe no disponibles/cargados.")
    if not glove_embeddings:
        print("[INFO] Específicamente, los embeddings de GloVe no se cargaron correctamente. Verifica el archivo y la ruta.")

[INFO] Cargando modelo GloVe desde: /content/glove.6B.50d.txt...
[SUCCESS] Modelo GloVe cargado. 400000 vectores de palabras.

--- Calculando Similitud de Palabras con Embeddings GloVe (Verbos más frecuentes) ---

--- Documento: doc1.txt ---
Verbo más común (lema): 'call'
  Los 5 términos más similares a 'call' según GloVe (coseno):
    - calls: 0.8872
    - calling: 0.8769
    - asking: 0.8592
    - ask: 0.8572
    - answer: 0.8378

--- Documento: doc2.txt ---
Verbo más común (lema): 'make'
  Los 5 términos más similares a 'make' según GloVe (coseno):
    - making: 0.9406
    - take: 0.9393
    - come: 0.9354
    - give: 0.9349
    - need: 0.9262

--- Documento: doc3.txt ---
Verbo más común (lema): 'write'
  Los 5 términos más similares a 'write' según GloVe (coseno):
    - writing: 0.8501
    - read: 0.8217
    - publish: 0.7848
    - notes: 0.7766
    - books: 0.7764

--- Documento: doc4.txt ---
Verbo más común (lema): 'present'
  Los 5 términos más similares a 'present' según GloVe

## 6. Similitud de documentos con “embedding” (10 pts)

Se realizará un procedimiento similar al del punto 4, pero esta vez utilizando el modelo **`bert-base-uncased`** para calcular la similitud entre las frases representativas.
BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje que genera embeddings contextuales, lo que significa que la representación de una palabra o frase depende de su contexto.

1.  Se utilizarán las frases representativas extraídas en el punto 4.
2.  Se obtendrá el embedding de cada frase utilizando `bert-base-uncased`.
3.  Se elegirá la frase del primer documento (`doc1.txt`) como base y se calculará la similitud coseno con las frases de los otros cuatro documentos.

In [11]:
try:
    print("[INFO] Cargando tokenizer y modelo BERT (bert-base-uncased)...")
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    bert_model.eval() # Poner el modelo en modo de evaluación (desactiva dropout, etc.)
    print("[SUCCESS] Tokenizer y modelo BERT cargados.")
except Exception as e:
    print(f"[ERROR] No se pudo cargar el modelo BERT: {e}")
    bert_tokenizer = None
    bert_model = None


def get_bert_sentence_embedding(sentence, tokenizer, model):
    """
    Obtiene el embedding de una frase usando un modelo BERT.
    El embedding se toma del token [CLS] de la última capa oculta.

    Args:
        sentence (str): La frase para la cual generar el embedding.
        tokenizer (BertTokenizer): El tokenizer de BERT.
        model (BertModel): El modelo BERT pre-entrenado.

    Returns:
        np.array: El vector de embedding de la frase, o None si hay error.
    """
    if not sentence.strip(): # Manejar frases vacías
        print("[WARNING] Se intentó obtener embedding BERT para una frase vacía.")
        return None
    if tokenizer is None or model is None:
        print("[ERROR] Tokenizer o modelo BERT no están disponibles para get_bert_sentence_embedding.")
        return None
    try:
        # Tokenizar la oración y añadir tokens especiales [CLS] y [SEP]
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)

        with torch.no_grad(): # Desactivar cálculo de gradientes para inferencia
            outputs = model(**inputs)

        # El embedding del token [CLS] se usa a menudo como representación de toda la secuencia.
        # outputs.last_hidden_state tiene forma (batch_size, sequence_length, hidden_size)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy() # Tomar el embedding [CLS] del primer (y único) item en el batch
        return cls_embedding
    except Exception as e:
        print(f"Error al generar embedding BERT para la frase '{sentence[:50]}...': {e}")
        return None

# --- Obtener embeddings BERT para las frases representativas ---
bert_embeddings_frases = {}
# Se verifica 'frases_representativas' aquí. Si no está definida, se imprimirá el mensaje del 'else' más abajo.
if 'frases_representativas' in locals() and frases_representativas and bert_tokenizer and bert_model:
    print("\n--- Generando Embeddings BERT para Frases Representativas ---")
    for file_id, frase in frases_representativas.items():
        if frase: # Solo si hay frase
            print(f"Procesando frase de '{file_id}'...")
            embedding = get_bert_sentence_embedding(frase, bert_tokenizer, bert_model)
            if embedding is not None:
                bert_embeddings_frases[file_id] = embedding
            else:
                 print(f"  No se pudo generar embedding para la frase de '{file_id}'.")
        else:
            print(f"  '{file_id}' no tiene frase representativa para procesar con BERT (frase vacía).")
else:
    print("\n[INFO] No se pueden generar embeddings BERT.")
    if 'frases_representativas' not in locals() or not frases_representativas:
        print("[INFO] La variable 'frases_representativas' no está definida o está vacía. Asegúrate de ejecutar la Sección 4.")
    if not bert_tokenizer or not bert_model:
        print("[INFO] El tokenizer o el modelo BERT no se cargaron correctamente.")


# --- Calcular similitud coseno entre embeddings BERT de las frases ---
if corpus and 'frases_representativas' in locals() and frases_representativas and bert_embeddings_frases and len(corpus.fileids()) > 1:
    print("\n--- Calculando Similitud de Frases con Embeddings BERT (Coseno) ---")

    ids_archivos = corpus.fileids()
    # Usaremos doc1.txt (índice 0) como base para la comparación,
    # igual que se hizo en el ejemplo de la práctica para BERT.
    id_base = ids_archivos[0]
    embedding_base_bert = bert_embeddings_frases.get(id_base)
    frase_base_texto = frases_representativas.get(id_base, "N/A")

    if embedding_base_bert is not None:
        print(f"Frase base (de '{id_base}'): \"{frase_base_texto}\"")
        for i in range(len(ids_archivos)): # Iterar sobre todos, incluyendo la comparación consigo mismo si se desea, o filtrar.
            id_comparar = ids_archivos[i]
            if id_comparar == id_base and i != 0: # Evitar procesar el base dos veces si está en otra posición (poco probable con fileids())
                continue
            if i == 0 and id_base != ids_archivos[0]: # Si id_base no fue el primero, asegurarse de procesar el primero.
                 # Este caso es más complejo de lo necesario si id_base siempre es ids_archivos[0]
                 pass


            embedding_comparar_bert = bert_embeddings_frases.get(id_comparar)
            frase_comparar_texto = frases_representativas.get(id_comparar, "N/A")

            if embedding_comparar_bert is not None:
                if id_comparar == id_base: # Comparación consigo mismo
                    similitud_bert = 1.0
                else:
                    # Similitud coseno = 1 - distancia coseno
                    similitud_bert = 1 - cosine(embedding_base_bert, embedding_comparar_bert)

                print(f"  Similitud con frase de '{id_comparar}' (\"{frase_comparar_texto[:50]}...\"): {similitud_bert:.4f}")
            else:
                # Solo imprimir si no es el documento base (ya que si el base no tiene embedding, se maneja antes)
                if id_comparar != id_base:
                    print(f"  No hay embedding BERT para la frase de '{id_comparar}'.")
    else:
        print(f"No se pudo obtener el embedding BERT para la frase base de '{id_base}'.")
elif not bert_embeddings_frases and ('frases_representativas' in locals() and frases_representativas):
    print("\n[INFO] No se calculará similitud con BERT porque no se generaron embeddings para las frases (ver mensajes anteriores).")
else:
    print("\n[INFO] No se puede calcular similitud de documentos con BERT. Corpus, frases representativas, o embeddings BERT no disponibles.")

[INFO] Cargando tokenizer y modelo BERT (bert-base-uncased)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[SUCCESS] Tokenizer y modelo BERT cargados.

--- Generando Embeddings BERT para Frases Representativas ---
Procesando frase de 'doc1.txt'...
Procesando frase de 'doc2.txt'...
Procesando frase de 'doc3.txt'...
Procesando frase de 'doc4.txt'...
Procesando frase de 'doc5.txt'...

--- Calculando Similitud de Frases con Embeddings BERT (Coseno) ---
Frase base (de 'doc1.txt'): "The date of the 'Jayamangla' is fixed between the tenth and thirteenth centuries A.D., because while treating of the sixty-four arts an example is taken from the 'Kávyaprakásha,' which was written about the tenth century A.D. Again, the copy of the commentary procured was evidently a transcript of a manuscript which once had a place in the library of a Chaulukyan king named Vishaladeva, a fact elicited from the following sentence at the end of it:—"
  Similitud con frase de 'doc1.txt' ("The date of the 'Jayamangla' is fixed between the ..."): 1.0000
  Similitud con frase de 'doc2.txt' ("By this is meant the desire on 

## Análisis y Conclusiones

Este experimento comparó diferentes enfoques para medir la similitud entre palabras y documentos (representados por frases). Se utilizaron WordNet (con `path_similarity` y `wup_similarity`), GloVe y BERT. Los documentos de entrada fueron introducciones de cinco libros del Proyecto Gutenberg con temáticas relacionadas con el amor, la sexualidad, y la literatura erótica/afectiva (Vatsyayana, Sacher-Masoch, Boccaccio, un texto sobre el "motivo amatorio" y pociones, y uno sobre educación sexual).

### Similitud de Palabras:

1.  **WordNet (`path_similarity` vs. `wup_similarity`):**
    *   **Tipo de palabras similares:** Ambas métricas arrojaron palabras que son semánticamente cercanas en la jerarquía de WordNet, principalmente sinónimos, hiperónimos (más generales) o hipónimos (más específicos). Por ejemplo, para 'call', ambas encontraron 'refer', 'dub', 'style', 'baptize'. Para 'work', ambas encontraron 'action', 'service', 'care'.
    *   **Diferencias en puntajes:** `wup_similarity` consistentemente produjo puntajes más altos que `path_similarity` para las mismas o similares parejas de palabras (e.g., 'call'-'refer': path=0.5000, wup=0.8571; 'work'-'action': path=0.5000, wup=0.9333). Esto se debe a que `wup_similarity` considera la profundidad de los synsets en la jerarquía y la profundidad de su ancestro común más específico (LCS). Si dos synsets son profundos (específicos) y su LCS también es profundo, `wup_similarity` les dará una alta puntuación, reflejando una cercanía conceptual específica. `path_similarity` se basa únicamente en la longitud de la ruta más corta, por lo que puede dar puntajes más bajos si la ruta es larga, incluso si los conceptos son conceptualmente cercanos y específicos.
    *   **Intuitividad:** `wup_similarity` a menudo parece más intuitiva para reflejar la "cercanía semántica" porque premia la especificidad compartida. `path_similarity` es más directa pero puede no capturar tan bien la cercanía entre conceptos muy específicos si están separados por varios nodos intermedios más generales.
    *   **Impacto de la estructura jerárquica:** Ambas métricas dependen fundamentalmente de la taxonomía de WordNet. `path_similarity` es una función inversa de la distancia en el grafo. `wup_similarity` utiliza la profundidad de los nodos y su LCS. Esto significa que los resultados están limitados por la estructura y el contenido de WordNet. Para 'boccaccio', WordNet lo relaciona con 'poet', 'bard', lo cual es correcto, mostrando su capacidad para manejar nombres propios si están en su lexicón como instancias de un tipo.

2.  **GloVe (similitud coseno):**
    *   **Tipo de palabras similares:** GloVe encontró palabras que tienden a co-ocurrir o usarse en contextos similares en su corpus de entrenamiento (Wikipedia + Gigaword). Estas pueden ser:
        *   Formas flexionadas o derivadas: 'call' -> 'calls', 'calling'; 'make' -> 'making'; 'write' -> 'writing'; 'give' -> 'giving'.
        *   Palabras temáticamente relacionadas o acciones/conceptos asociados: 'call' -> 'asking', 'ask', 'answer'; 'make' -> 'take', 'come', 'give', 'need'; 'write' -> 'read', 'publish', 'notes', 'books'.
        *   Un caso particular fue para 'present' (verbo más común en doc4.txt, aunque solo con frecuencia 2), donde GloVe devolvió 'same', 'which', 'of', 'there', 'however'. Esto sugiere que la palabra 'present' en su forma base es muy común y co-ocurre con una amplia gama de palabras funcionales o estructurales en el vasto corpus de GloVe, y el embedding puede haber capturado este uso más general o su rol sintáctico en lugar de un significado semántico específico de "presentar algo".
    *   **Comparación con WordNet:**
        *   WordNet ofrece similitud basada en relaciones léxicas curadas (sinonimia, hiperonimia). GloVe ofrece similitud basada en distribución y contexto.
        *   Para 'call', WordNet dio sinónimos de acción ('refer', 'dub'), mientras GloVe dio formas de la palabra y acciones relacionadas ('asking', 'answer').
        *   Para 'write', WordNet dio acciones similares ('draw', 'verse'), GloVe dio elementos del ecosistema de la escritura ('read', 'publish', 'books').
        *   GloVe captura aspectos diferentes de la similitud, más asociativos y contextuales, mientras WordNet es más taxonómico.
    *   **Impacto del corpus de entrenamiento:** Los resultados de GloVe reflejan las asociaciones presentes en Wikipedia y Gigaword. Si el corpus fuera diferente (ej., literatura médica), las palabras similares también lo serían. El caso de 'present' es un buen ejemplo de cómo un corpus general puede llevar a asociaciones muy amplias para palabras comunes.

3.  **Limitaciones Observadas:**
    *   **WordNet:**
        *   *Cobertura de vocabulario:* Aunque bueno, no es exhaustivo. Neologismos, jerga o términos muy especializados pueden faltar. 'boccaccio' fue encontrado, pero otros nombres propios o términos técnicos podrían no estar.
        *   *Sensibilidad al contexto (limitada):* Aunque se usa POS tagging, WordNet opera principalmente a nivel de lema y su "sentido más común" o el primer synset. No captura el matiz contextual fino que una palabra puede tener en una frase específica.
        *   *Dependencia de la calidad del preprocesamiento:* Errores en POS tagging o lematización afectan directamente la búsqueda en WordNet.
    *   **GloVe:**
        *   *Palabras Fuera de Vocabulario (OOV):* Aunque el modelo `glove.6B` tiene un vocabulario grande (400k), aún puede haber palabras OOV. Los verbos frecuentes analizados estaban presentes.
        *   *Polisemia:* GloVe asigna un único vector a cada palabra (ej., "bank"). Si una palabra tiene múltiples significados, su vector es una mezcla de todos sus contextos, lo que puede llevar a resultados de similitud menos precisos para un sentido particular.
        *   *Naturaleza de la similitud:* Como se vio con 'present', la similitud coseno puede reflejar co-ocurrencia frecuente con palabras funcionales más que una similitud semántica profunda para palabras muy comunes y polisémicas.

### Similitud de Documentos (Frases Representativas):

Se comparó una frase base con las frases representativas de los otros documentos.

1.  **WordNet (`document_path_similarity`):**
    *   **Interpretación de puntajes:** Se usó la frase de `doc5.txt` ("We give to young folks... sex education...") como base.
        *   vs `doc2.txt` (Sacher-Masoch, "...desire... subject to will..."): 0.2869 (la más alta, lo cual es coherente ya que ambos tratan temas de sexualidad, aunque desde ángulos muy diferentes).
        *   vs `doc3.txt` (Boccaccio): 0.2379 (la más baja, lo que tiene sentido ya que Boccaccio es más literario/histórico).
        *   vs `doc4.txt` (amatory motif, pociones): 0.2389 (también baja, aunque habla de amor/deseo, es más abstracto/místico).
    *   Los puntajes son relativamente bajos en general, lo cual es común cuando se promedian similitudes de `path_similarity` entre muchos synsets. Reflejan una similitud temática general, pero la señal puede ser débil.
    *   **Impacto de la agregación:** El método promedia las mejores similitudes de synset a synset. Esto puede ser una simplificación excesiva, ya que pierde la estructura sintáctica de la frase y el significado composicional. La similitud general puede ser influenciada desproporcionadamente por unas pocas palabras clave muy similares o diluida por muchas palabras no coincidentes.

2.  **BERT (similitud coseno sobre embeddings [CLS]):**
    *   **Comparación con WordNet:** Se usó la frase de `doc1.txt` ("The date of the 'Jayamangla'...") como base.
        *   vs `doc5.txt` (sex education): 0.7806 (la más alta). Doc1 (Vatsyayana/Kama Sutra) y doc5 (educación sexual) son temáticamente los más cercanos.
        *   vs `doc2.txt` (Sacher-Masoch): 0.7630 (segunda más alta). También temáticamente relevante.
        *   vs `doc4.txt` (amatory motif): 0.7457 (tercera). Relacionado con amor/deseo.
        *   vs `doc3.txt` (Boccaccio): 0.6534 (la más baja). Coherente, al ser el más distante temáticamente (literario/narrativo).
    *   Los puntajes de BERT son significativamente más altos y parecen ofrecer una discriminación más clara y semánticamente más coherente entre las frases.
    *   **Reflejo del contexto:** BERT captura el contexto de manera bidireccional. El embedding [CLS] se considera una representación de toda la frase. Esto se reflejó en una evaluación de similitud que parece más alineada con la comprensión humana de las frases. El orden de similitud obtenido con BERT para `doc1.txt` es muy intuitivo dada la temática de los libros.
    *   **Discrepancias:** Aunque las frases base fueron diferentes, la *calidad* de la clasificación de similitud de BERT parece superior. BERT probablemente identificaría patrones de similitud más robustos independientemente de la frase base elegida, siempre que las frases sean temáticamente distintas.

3.  **Impacto de la Extracción de Frases (TextRank):**
    *   Las frases extraídas por TextRank parecían ser resúmenes razonables de las introducciones. La calidad de esta frase es crucial. Si la frase extraída no es representativa, la comparación de similitud (para cualquier método) será defectuosa. Una frase más larga y detallada podría beneficiar a BERT, mientras que una frase con palabras clave claras podría ser suficiente para el enfoque de WordNet basado en synsets. En general, las frases eran lo suficientemente largas y distintivas para permitir una comparación significativa.

### General:

*   **Robustez/Utilidad:**
    *   *Similitud de palabras:* WordNet es útil para encontrar relaciones léxicas explícitas (sinónimos, hiperónimos). GloVe es bueno para similitud asociativa/contextual. La elección depende del objetivo.
    *   *Similitud de documentos:* BERT fue claramente más robusto y útil. Su capacidad para entender el contexto de la frase completa supera al enfoque de WordNet basado en la agregación de similitudes de palabras individuales.
*   **Ventajas y Desventajas:**
    *   **WordNet (Conocimiento Léxico):**
        *   *Ventajas:* Relaciones semánticas explícitas y curadas, interpretable, no requiere entrenamiento (el recurso ya existe).
        *   *Desventajas:* Cobertura limitada, estático (no se adapta a nuevos usos del lenguaje), sensible a la calidad del preprocesamiento (POS, lematización), dificultad para capturar significado composicional y contextual complejo.
    *   **Embeddings (GloVe, BERT - Basados en Datos):**
        *   *Ventajas:* Aprenden de grandes cantidades de texto, capturan matices y asociaciones no explícitas en lexicón. BERT, en particular, maneja bien el contexto y la polisemia (a nivel de frase).
        *   *Desventajas:* Pueden ser "cajas negras" (difícil interpretar *por qué* dos cosas son similares). Requieren grandes datos y recursos computacionales para entrenamiento (aunque aquí usamos pre-entrenados). GloVe tiene un vector por palabra, lo que dificulta la polisemia. OOV puede ser un problema (menos para BERT debido a sub-palabras).

**Desafíos Encontrados y Coherencia Temática:**
*   El tema de los libros (amor, sexualidad, erotismo) proporcionó un buen campo de pruebas. Las similitudes encontradas, especialmente con BERT para documentos y con WordNet/GloVe para palabras como 'sex', 'potion', 'write', fueron en general coherentes con estas temáticas.
*   El caso del verbo 'present' con GloVe fue un desafío interpretativo, mostrando que la "similitud" de GloVe no siempre es semántica directa, sino contextual amplia.
*   La elección del "primer synset" en WordNet es una simplificación; un enfoque más sofisticado podría intentar desambiguar el sentido de la palabra en contexto antes de buscar similitudes, pero esto añade complejidad.

En conclusión, los métodos basados en embeddings, y BERT en particular, demuestran una capacidad superior para capturar la similitud semántica a nivel de frase/documento debido a su comprensión contextual. WordNet sigue siendo valioso para analizar relaciones léxicas explícitas entre palabras individuales. GloVe ofrece un punto intermedio, capturando asociaciones contextuales a nivel de palabra. La elección de la herramienta depende de la tarea específica y del tipo de similitud que se desea medir.